# List of Exposures  at USDF


- work with Weakly_2023_35
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2023/09/15



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTel_summit_tests_2023/butlertools', '/home/d/dagoret/repos/repos_w_2023_35/analysis_tools/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/cp_pipe/g857a3418e4+7df23b908a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/shapelet/g44018dc512+ea6441928c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_modelfit/g93c4d6e787+79b822255a/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/proxmin/g33b4157f25/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet/g63ff1f842d+4083830bf8/lib/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_scarlet/g943cb464a2+f79590064d/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/psfex/gc

In [3]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2023_35/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_35/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_35/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2023_35
   g4213664e8e+c55b99ec95 	current w_2023_35 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  day_obs: int
  seq_num: int
  seq_start: int
  seq_end: int
  group_name: str
  group_id: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    
    df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2022083100004
  physical_filter: 'unknown~unknown'
  obs_id: 'AT_O_20220831_000004'
  exposure_time: 0.0
  dark_time: 0.0160482
  observation_type: 'bias'
  observation_reason: 'bias'
  day_obs: 20220831
  seq_num: 4
  seq_start: 4
  seq_end: 4
  group_name: '2022083100004'
  group_id: 2022083100004
  target_name: 'UNKNOWN'
  science_program: 'unknown'
  tracking_ra: None
  tracking_dec: None
  sky_angle: None
  azimuth: None
  zenith_angle: None
  has_simulated: False
  timespan: Timespan(begin=astropy.time.Time('2022-08-31 22:58:13.633984', scale='tai', format='iso'), end=astropy.time.Time('2022-08-31 22:58:13.650000', scale='tai', format='iso'))
	 id:                   2022083100004
	 day_obs:              20220831
	 seq_num:              4
	 type-of-observation:  bias
	 target:               UNKNOWN
59822.957102245186 2459823.45710

In [14]:
info.toDict()

{'instrument': 'LATISS',
 'id': 2023091400251,
 'physical_filter': 'empty~holo4_003',
 'obs_id': 'AT_O_20230914_000251',
 'exposure_time': 30.0,
 'dark_time': 30.2412,
 'observation_type': 'science',
 'observation_reason': 'object',
 'day_obs': 20230914,
 'seq_num': 251,
 'seq_start': 251,
 'seq_end': 251,
 'group_name': '2023-09-15T04:39:37.384',
 'group_id': 2746823773840000,
 'target_name': 'HD185975',
 'science_program': 'SITCOM-1001',
 'tracking_ra': 307.6443361306818,
 'tracking_dec': -87.51340661010596,
 'sky_angle': 139.50119907932705,
 'azimuth': 182.18586270039003,
 'zenith_angle': 57.98427685321544,
 'has_simulated': False,
 'timespan': Timespan(begin=astropy.time.Time('2023-09-15 04:54:48.783999', scale='tai', format='iso'), end=astropy.time.Time('2023-09-15 04:55:19.025000', scale='tai', format='iso'))}

In [15]:
info

exposure.RecordClass(instrument='LATISS', id=2023091400251, physical_filter='empty~holo4_003', obs_id='AT_O_20230914_000251', exposure_time=30.0, dark_time=30.2412, observation_type='science', observation_reason='object', day_obs=20230914, seq_num=251, seq_start=251, seq_end=251, group_name='2023-09-15T04:39:37.384', group_id=2746823773840000, target_name='HD185975', science_program='SITCOM-1001', tracking_ra=307.6443361306818, tracking_dec=-87.51340661010596, sky_angle=139.50119907932705, azimuth=182.18586270039003, zenith_angle=57.98427685321544, has_simulated=False, timespan=Timespan(begin=astropy.time.Time('2023-09-15 04:54:48.783999', scale='tai', format='iso'), end=astropy.time.Time('2023-09-15 04:55:19.025000', scale='tai', format='iso')))

In [16]:
info.azimuth

182.18586270039003

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58407,2023091400239,AT_O_20230914_000239,20230914,239,2023-09-15 04:46:39.180016,2023-09-15 04:47:09.410,science,HD205905,collimator~holo4_003,25.986033,30.0,324.773662,-27.348941,193.540132,269.242833,25.986033,SITCOM-1001,2.460203e+06,60202.199065
58408,2023091400242,AT_O_20230914_000242,20230914,242,2023-09-15 04:48:46.671982,2023-09-15 04:49:06.914,science,HD205905,empty~blue300lpmm_qn1,26.444155,20.0,324.773780,-27.348939,193.540051,268.989976,26.444155,SITCOM-1001,2.460203e+06,60202.200540
58409,2023091400245,AT_O_20230914_000245,20230914,245,2023-09-15 04:51:28.727993,2023-09-15 04:51:33.956,acq,HD185975,empty~holo4_003,57.954913,5.0,307.672800,-87.511735,139.501997,182.156343,57.954913,SITCOM-1001,2.460203e+06,60202.202416
58410,2023091400248,AT_O_20230914_000248,20230914,248,2023-09-15 04:52:38.951993,2023-09-15 04:53:09.191,science,HD185975,empty~holo4_001,57.966601,30.0,307.644804,-87.513430,139.500983,182.166499,57.966601,SITCOM-1001,2.460203e+06,60202.203229


In [18]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [19]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2022083100004,AT_O_20220831_000004,20220831,4,2022-08-31 22:58:13.633984,2022-08-31 22:58:13.650,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957102
1,2022083100008,AT_O_20220831_000008,20220831,8,2022-08-31 22:59:16.387013,2022-08-31 22:59:16.398,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957829
2,2022083100007,AT_O_20220831_000007,20220831,7,2022-08-31 22:59:11.978990,2022-08-31 22:59:11.991,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957778
3,2022083100009,AT_O_20220831_000009,20220831,9,2022-08-31 22:59:20.801996,2022-08-31 22:59:20.813,bias,UNKNOWN,unknown~unknown,NaN,0.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.957880
4,2022083100012,AT_O_20220831_000012,20220831,12,2022-08-31 23:00:00.113994,2022-08-31 23:00:03.124,dark,UNKNOWN,unknown~unknown,NaN,3.0,NaN,NaN,NaN,NaN,NaN,unknown,2.459823e+06,59822.958335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58407,2023091400239,AT_O_20230914_000239,20230914,239,2023-09-15 04:46:39.180016,2023-09-15 04:47:09.410,science,HD205905,collimator~holo4_003,25.986033,30.0,324.773662,-27.348941,193.540132,269.242833,25.986033,SITCOM-1001,2.460203e+06,60202.199065
58408,2023091400242,AT_O_20230914_000242,20230914,242,2023-09-15 04:48:46.671982,2023-09-15 04:49:06.914,science,HD205905,empty~blue300lpmm_qn1,26.444155,20.0,324.773780,-27.348939,193.540051,268.989976,26.444155,SITCOM-1001,2.460203e+06,60202.200540
58409,2023091400245,AT_O_20230914_000245,20230914,245,2023-09-15 04:51:28.727993,2023-09-15 04:51:33.956,acq,HD185975,empty~holo4_003,57.954913,5.0,307.672800,-87.511735,139.501997,182.156343,57.954913,SITCOM-1001,2.460203e+06,60202.202416
58410,2023091400248,AT_O_20230914_000248,20230914,248,2023-09-15 04:52:38.951993,2023-09-15 04:53:09.191,science,HD185975,empty~holo4_001,57.966601,30.0,307.644804,-87.513430,139.500983,182.166499,57.966601,SITCOM-1001,2.460203e+06,60202.203229


## Selection of science

In [20]:
df_science = df_exposure[df_exposure.type == 'science']

In [21]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_8923/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [22]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
58287,2023091400165,AT_O_20230914_000165,20230914,165,2023-09-15 03:48:38.855019,2023-09-15 03:49:09.094,science,HD009051,empty~holo4_001,37.938148,30.0,22.188402,-24.292704,341.384834,91.316090,37.938148,SITCOM-1001,2.460203e+06,60202.158783
58317,2023091400166,AT_O_20230914_000166,20230914,166,2023-09-15 03:49:21.858004,2023-09-15 03:49:52.095,science,HD009051,empty~holo4_001,37.786620,30.0,22.188376,-24.292699,341.384763,91.232017,37.786620,SITCOM-1001,2.460203e+06,60202.159281
58347,2023091400167,AT_O_20230914_000167,20230914,167,2023-09-15 03:50:05.992989,2023-09-15 03:50:36.221,science,HD009051,empty~holo4_003,37.624417,30.0,22.188508,-24.292640,341.381335,91.141637,37.624417,SITCOM-1001,2.460203e+06,60202.159792
58288,2023091400168,AT_O_20230914_000168,20230914,168,2023-09-15 03:50:48.934016,2023-09-15 03:51:19.171,science,HD009051,empty~holo4_003,37.469225,30.0,22.188438,-24.292663,341.384802,91.055016,37.469225,SITCOM-1001,2.460203e+06,60202.160289
58318,2023091400169,AT_O_20230914_000169,20230914,169,2023-09-15 03:51:33.695994,2023-09-15 03:52:03.936,science,HD009051,collimator~holo4_003,37.310483,30.0,22.188433,-24.292666,341.384832,90.966085,37.310483,SITCOM-1001,2.460203e+06,60202.160807
58348,2023091400170,AT_O_20230914_000170,20230914,170,2023-09-15 03:52:16.803988,2023-09-15 03:52:47.050,science,HD009051,collimator~holo4_003,37.151735,30.0,22.188426,-24.292673,341.384763,90.876856,37.151735,SITCOM-1001,2.460203e+06,60202.161306
58289,2023091400171,AT_O_20230914_000171,20230914,171,2023-09-15 03:53:07.441009,2023-09-15 03:53:27.671,science,HD009051,empty~blue300lpmm_qn1,36.971333,20.0,22.188411,-24.292656,341.384787,90.775042,36.971333,SITCOM-1001,2.460203e+06,60202.161892
58319,2023091400172,AT_O_20230914_000172,20230914,172,2023-09-15 03:53:40.450003,2023-09-15 03:54:00.683,science,HD009051,empty~blue300lpmm_qn1,36.852292,20.0,22.188439,-24.292669,341.384824,90.707679,36.852292,SITCOM-1001,2.460203e+06,60202.162274
58349,2023091400173,AT_O_20230914_000173,20230914,173,2023-09-15 03:54:13.458998,2023-09-15 03:54:33.702,science,HD009051,empty~blue300lpmm_qn1,36.733200,20.0,22.188419,-24.292704,341.384821,90.640150,36.733200,SITCOM-1001,2.460203e+06,60202.162656
58321,2023091400178,AT_O_20230914_000178,20230914,178,2023-09-15 03:57:28.542993,2023-09-15 03:57:58.787,science,HD185975,empty~holo4_001,57.559159,30.0,307.889338,-87.505915,125.417277,181.601069,57.559159,SITCOM-1001,2.460203e+06,60202.164914


In [23]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720, 20230801, 20230802, 20230803, 20230815, 20230816,
       20230817, 20230829, 20230912, 20230913, 20230914])

## Selection of date

2021-02-16  2021-02-18  2021-06-08  2021-06-10  2021-07-07  2021-09-07  2021-09-09  2021-10-06  2021-11-02  2021-11-04  2022-02-16  2022-03-15  2022-03-17  20220502  
2021-02-17  2021-03-11  2021-06-09  2021-07-06  2021-07-08  2021-09-08  2021-10-05  2021-10-07  2021-11-03  2022-02-15  2022-02-17  2022-03-16  2022-03-18  20220503 20220524 20220607 20220608 20220609

### Add those

20220504 - Not existing

In [24]:
#date_sel = 20220630
#date_sel = 20220630
#date_sel = 20221208
#date_sel = 20221209

# data in january
#date_sel= 20230118
#date_sel= 20230131

# data in February
#date_sel = 20230201
#date_sel = 20230202
#date_sel = 20230704

#date_sel = 20230510
#date_sel = 20230509
#date_sel = 20230511

#date_sel = 20230705
#date_sel = 20230706

#july 2023
#date_sel = 20230718
#date_sel = 20230720

#august 2023
#date_sel = 20230801
#date_sel = 20230802

#date_sel = 20230815
#date_sel = 20230817


#september 2023
#date_sel = 20230912
#date_sel = 20230913
date_sel = 20230914



## Query Dimension records on Exposures

In [25]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [26]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [27]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [28]:
len(df_science_selected)

165

# List of filters

In [29]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~empty', 'collimator~holo4_003', 'cyl_lens~holo4_003',
       'cyl_lens~blue300lpmm_qn1', 'SDSSr_65mm~empty', 'empty~holo4_001',
       'empty~holo4_003', 'empty~blue300lpmm_qn1'], dtype=object)

In [30]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['collimator~holo4_003', 'cyl_lens~holo4_003', 'empty~holo4_001',
       'empty~holo4_003'], dtype='<U20')

## Selection of filter

- But now we have a loop on filters

In [31]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

collimator~holo4_003
visitdispersers_20230914_filt_collimator-holo4_003.list
cyl_lens~holo4_003
visitdispersers_20230914_filt_cyl_lens-holo4_003.list
empty~holo4_001
visitdispersers_20230914_filt_empty-holo4_001.list
empty~holo4_003
visitdispersers_20230914_filt_empty-holo4_003.list


/tmp/ipykernel_8923/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_8923/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
/tmp/ipykernel_8923/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [34]:
N = len(all_def_science_selected_final)

In [35]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
58240,2023091400026,AT_O_20230914_000026,20230914,26,2023-09-15 00:47:01.206987,2023-09-15 00:52:01.454,science,PN M 3-38,collimator~holo4_003,29.391281,300.0,260.284602,-29.006504,0.001767,263.711026,29.391281,SITCOM-1022,2.460203e+06,60202.032653
58183,2023091400027,AT_O_20230914_000027,20230914,27,2023-09-15 00:52:14.142002,2023-09-15 00:57:14.379,science,PN M 3-38,collimator~holo4_003,30.513444,300.0,260.284589,-29.006496,0.001757,263.274444,30.513444,SITCOM-1022,2.460203e+06,60202.036275
58234,2023091400094,AT_O_20230914_000094,20230914,94,2023-09-15 02:45:30.628011,2023-09-15 02:46:00.859,science,HD160617,collimator~holo4_003,47.662725,30.0,265.689633,-40.369394,186.268917,241.093191,47.662725,SITCOM-1001,2.460203e+06,60202.114938
58263,2023091400095,AT_O_20230914_000095,20230914,95,2023-09-15 02:46:18.073981,2023-09-15 02:46:48.337,science,HD160617,collimator~holo4_003,47.814317,30.0,265.689647,-40.369205,186.269040,241.068527,47.814317,SITCOM-1001,2.460203e+06,60202.115487
58210,2023091400108,AT_O_20230914_000108,20230914,108,2023-09-15 02:58:39.078009,2023-09-15 02:59:09.332,science,HD160617,collimator~holo4_003,50.150913,30.0,265.687745,-40.368151,188.367038,240.642574,50.150913,SITCOM-1001,2.460203e+06,60202.124063
58239,2023091400109,AT_O_20230914_000109,20230914,109,2023-09-15 02:59:18.764014,2023-09-15 02:59:48.993,science,HD160617,collimator~holo4_003,50.273534,30.0,265.687690,-40.368369,188.366973,240.617678,50.273534,SITCOM-1001,2.460203e+06,60202.124523
58272,2023091400120,AT_O_20230914_000120,20230914,120,2023-09-15 03:06:43.715010,2023-09-15 03:07:13.954,science,HD185975,collimator~holo4_003,57.298562,30.0,308.040310,-87.496931,111.674854,180.990492,57.298562,SITCOM-1001,2.460203e+06,60202.129673
58302,2023091400121,AT_O_20230914_000121,20230914,121,2023-09-15 03:07:26.770017,2023-09-15 03:07:57.005,science,HD185975,collimator~holo4_003,57.301262,30.0,308.042309,-87.496971,111.672669,180.999440,57.301262,SITCOM-1001,2.460203e+06,60202.130171
58276,2023091400132,AT_O_20230914_000132,20230914,132,2023-09-15 03:15:59.739996,2023-09-15 03:16:29.978,science,HD160617,collimator~holo4_003,53.410883,30.0,265.685937,-40.368316,190.653671,239.917756,53.410883,SITCOM-1001,2.460203e+06,60202.136108
58306,2023091400133,AT_O_20230914_000133,20230914,133,2023-09-15 03:16:42.659981,2023-09-15 03:17:12.895,science,HD160617,collimator~holo4_003,53.545108,30.0,265.685905,-40.368317,190.653597,239.884863,53.545108,SITCOM-1001,2.460203e+06,60202.136605


In [36]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
58213,2023091400031,AT_O_20230914_000031,20230914,31,2023-09-15 01:12:25.000009,2023-09-15 01:14:25.241,science,zet Sgr,cyl_lens~holo4_003,12.902522,120.0,285.655397,-29.882538,185.324410,267.994034,12.902522,SITCOM-1009,2.460203e+06,60202.050289
58242,2023091400032,AT_O_20230914_000032,20230914,32,2023-09-15 01:14:38.005001,2023-09-15 01:17:38.245,science,zet Sgr,cyl_lens~holo4_003,13.382216,180.0,285.655311,-29.882547,185.324446,267.789677,13.382216,SITCOM-1009,2.460203e+06,60202.051829
58185,2023091400033,AT_O_20230914_000033,20230914,33,2023-09-15 01:17:53.426995,2023-09-15 01:21:53.664,science,zet Sgr,cyl_lens~holo4_003,14.085283,240.0,285.655332,-29.882527,185.324449,267.497765,14.085283,SITCOM-1009,2.460203e+06,60202.054091
58214,2023091400034,AT_O_20230914_000034,20230914,34,2023-09-15 01:22:06.448998,2023-09-15 01:27:06.698,science,zet Sgr,cyl_lens~holo4_003,14.997183,300.0,285.655414,-29.882545,185.324316,267.130224,14.997183,SITCOM-1009,2.460203e+06,60202.057019
58243,2023091400035,AT_O_20230914_000035,20230914,35,2023-09-15 01:27:47.135020,2023-09-15 01:29:47.362,science,zet Sgr,cyl_lens~holo4_003,16.226019,120.0,285.655276,-29.882529,185.324442,266.651440,16.226019,SITCOM-1009,2.460203e+06,60202.060962
58186,2023091400036,AT_O_20230914_000036,20230914,36,2023-09-15 01:30:00.259987,2023-09-15 01:33:00.506,science,zet Sgr,cyl_lens~holo4_003,16.705045,180.0,285.655335,-29.882528,185.324464,266.468997,16.705045,SITCOM-1009,2.460203e+06,60202.062503
58215,2023091400037,AT_O_20230914_000037,20230914,37,2023-09-15 01:33:13.301989,2023-09-15 01:37:13.532,science,zet Sgr,cyl_lens~holo4_003,17.400084,240.0,285.655332,-29.882516,185.324436,266.207871,17.400084,SITCOM-1009,2.460203e+06,60202.064737
58244,2023091400038,AT_O_20230914_000038,20230914,38,2023-09-15 01:37:26.388003,2023-09-15 01:42:26.636,science,zet Sgr,cyl_lens~holo4_003,18.310889,300.0,285.655313,-29.882513,185.324531,265.871180,18.310889,SITCOM-1009,2.460203e+06,60202.067667


In [37]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
58204,2023091400090,AT_O_20230914_000090,20230914,90,2023-09-15 02:42:24.434009,2023-09-15 02:42:54.668,science,HD160617,empty~holo4_001,47.074960,30.0,265.689924,-40.367975,186.270185,241.188421,47.074960,SITCOM-1001,2.460203e+06,60202.112783
58233,2023091400091,AT_O_20230914_000091,20230914,91,2023-09-15 02:43:12.123994,2023-09-15 02:43:42.353,science,HD160617,empty~holo4_001,47.226673,30.0,265.689939,-40.367932,186.270133,241.164911,47.226673,SITCOM-1001,2.460203e+06,60202.113335
58236,2023091400100,AT_O_20230914_000100,20230914,100,2023-09-15 02:51:58.279982,2023-09-15 02:52:28.522,science,HD160617,empty~holo4_001,48.888060,30.0,265.688139,-40.369206,187.838225,240.882094,48.888060,SITCOM-1001,2.460203e+06,60202.119425
58266,2023091400104,AT_O_20230914_000104,20230914,104,2023-09-15 02:55:47.976020,2023-09-15 02:56:18.210,science,HD160617,empty~holo4_001,49.612982,30.0,265.687620,-40.368608,188.366906,240.747405,49.612982,SITCOM-1001,2.460203e+06,60202.122083
58209,2023091400105,AT_O_20230914_000105,20230914,105,2023-09-15 02:56:27.521008,2023-09-15 02:56:57.756,science,HD160617,empty~holo4_001,49.735731,30.0,265.687672,-40.368300,188.367240,240.724119,49.735731,SITCOM-1001,2.460203e+06,60202.122541
58330,2023091400116,AT_O_20230914_000116,20230914,116,2023-09-15 03:03:48.947018,2023-09-15 03:04:19.184,science,HD185975,empty~holo4_001,57.287917,30.0,308.035968,-87.496860,111.679668,180.953952,57.287917,SITCOM-1001,2.460203e+06,60202.127650
58271,2023091400117,AT_O_20230914_000117,20230914,117,2023-09-15 03:04:31.959981,2023-09-15 03:05:02.188,science,HD185975,empty~holo4_001,57.290510,30.0,308.040499,-87.496935,111.674886,180.962806,57.290510,SITCOM-1001,2.460203e+06,60202.128148
58334,2023091400128,AT_O_20230914_000128,20230914,128,2023-09-15 03:13:04.341994,2023-09-15 03:13:34.581,science,HD160617,empty~holo4_001,52.864369,30.0,265.685791,-40.368314,190.654007,240.049231,52.864369,SITCOM-1001,2.460203e+06,60202.134078
58275,2023091400129,AT_O_20230914_000129,20230914,129,2023-09-15 03:13:47.351014,2023-09-15 03:14:17.579,science,HD160617,empty~holo4_001,52.998644,30.0,265.685929,-40.368336,190.653760,240.017268,52.998644,SITCOM-1001,2.460203e+06,60202.134576
58279,2023091400141,AT_O_20230914_000141,20230914,141,2023-09-15 03:29:38.514996,2023-09-15 03:30:08.746,science,HD167060,empty~holo4_001,49.750493,30.0,274.439308,-61.757257,169.418310,212.603898,49.750493,SITCOM-1001,2.460203e+06,60202.145585


In [38]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
58262,2023091400092,AT_O_20230914_000092,20230914,92,2023-09-15 02:43:57.308018,2023-09-15 02:44:27.542,science,HD160617,empty~holo4_003,47.368864,30.0,265.689955,-40.367964,186.270079,241.142454,47.368864,SITCOM-1001,2.460203e+06,60202.113858
58205,2023091400093,AT_O_20230914_000093,20230914,93,2023-09-15 02:44:43.079003,2023-09-15 02:45:13.326,science,HD160617,empty~holo4_003,47.514216,30.0,265.689746,-40.369115,186.269086,241.117645,47.514216,SITCOM-1001,2.460203e+06,60202.114387
58238,2023091400106,AT_O_20230914_000106,20230914,106,2023-09-15 02:57:07.284019,2023-09-15 02:57:37.528,science,HD160617,empty~holo4_003,49.861507,30.0,265.687809,-40.368160,188.367120,240.699801,49.861507,SITCOM-1001,2.460203e+06,60202.123001
58267,2023091400107,AT_O_20230914_000107,20230914,107,2023-09-15 02:57:58.834011,2023-09-15 02:58:29.061,science,HD160617,empty~holo4_003,50.021920,30.0,265.687787,-40.368349,188.366908,240.667972,50.021920,SITCOM-1001,2.460203e+06,60202.123598
58301,2023091400118,AT_O_20230914_000118,20230914,118,2023-09-15 03:05:16.012006,2023-09-15 03:05:46.253,science,HD185975,empty~holo4_003,57.293183,30.0,308.042752,-87.496967,111.672499,180.971989,57.293183,SITCOM-1001,2.460203e+06,60202.128658
58331,2023091400119,AT_O_20230914_000119,20230914,119,2023-09-15 03:05:58.989000,2023-09-15 03:06:29.221,science,HD185975,empty~holo4_003,57.295832,30.0,308.032627,-87.496799,111.682815,180.981635,57.295832,SITCOM-1001,2.460203e+06,60202.129155
58305,2023091400130,AT_O_20230914_000130,20230914,130,2023-09-15 03:14:32.104986,2023-09-15 03:15:02.337,science,HD160617,empty~holo4_003,53.139281,30.0,265.685862,-40.368321,190.653841,239.983581,53.139281,SITCOM-1001,2.460203e+06,60202.135094
58335,2023091400131,AT_O_20230914_000131,20230914,131,2023-09-15 03:15:15.102017,2023-09-15 03:15:45.339,science,HD160617,empty~holo4_003,53.273572,30.0,265.685860,-40.368323,190.653769,239.951151,53.273572,SITCOM-1001,2.460203e+06,60202.135591
58339,2023091400143,AT_O_20230914_000143,20230914,143,2023-09-15 03:31:05.664996,2023-09-15 03:31:35.900,science,HD167060,empty~holo4_003,49.919734,30.0,274.439232,-61.757220,169.418398,212.643975,49.919734,SITCOM-1001,2.460203e+06,60202.146593
58280,2023091400144,AT_O_20230914_000144,20230914,144,2023-09-15 03:31:48.687012,2023-09-15 03:32:18.922,science,HD167060,empty~holo4_003,50.003345,30.0,274.439422,-61.757222,169.418065,212.663205,50.003345,SITCOM-1001,2.460203e+06,60202.147091


In [39]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [40]:
df_forsave = pd.concat(all_def_science_selected_final)

In [41]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [42]:
filename_out_csv

'holosummary_20230914.csv'

In [43]:
df_forsave.to_csv(filename_out_csv) 

In [44]:
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [45]:
df_forsave.to_excel(filename_out_excel) 